In [10]:
pip install ipadic

Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install fugashi

Note: you may need to restart the kernel to use updated packages.


In [7]:
!pip install mecab-python3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.5/314.5 kB 5.0 MB/s eta 0:00:00a 0:00:01


In [8]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 707.4 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 kB 3.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.6/188.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 715.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 1.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import torch
from torch.optim import AdamW
from transformers import pipeline, AutoModelForSequenceClassification, BertJapaneseTokenizer

/Users/masanobu/miniconda3/envs/test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('sugakubunka.csv')
df.dropna(inplace=True)
df = df.drop(["Unnamed: 0","text_cleaned"],axis=1)
df["label"] = df["label"].astype(int)
df = df.replace({0:'NEGATIVE', 1:'POSITIVE'})
df

,text,label
0,サンプルコード見ながらバグってる,NEGATIVE
1,python多めの後にR触るとdata.trainとか書いてあるとき頭がバグる,NEGATIVE
7,痩せる,POSITIVE
8,ホームページの写真、新しくしてもらったんだけど顔丸すぎて萎えた,NEGATIVE
9,はい！再来週やります！\n無料なのでお気軽にご参加ください😊 https://t.co/Vf...,POSITIVE
...,...,...
172,肉眼ならいい感じに見えたのになー,POSITIVE
173,都会の月食(うるとらわかりにくい) https://t.co/5wIeonJexx,NEGATIVE
174,復習します！,POSITIVE
175,講師勉強会、今日はとてもむじゅかしぃでした\n(ミュート外れてるのに気付かずに独り言聞かれた人),POSITIVE


In [3]:
train_docs = df["text"].tolist()
train_labels = df["label"].tolist()

In [13]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, id2label=id2label, label2id=label2id)
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [14]:
encodings = tokenizer(train_docs, return_tensors='pt', padding=True, truncation=True, max_length=128)
input_ids = encodings['input_ids']
attention_mask = encodings['attention_mask']

In [15]:
# Fine-tuning in native PyTorch

# the AdamW() optimizer which implements gradient bias correction as well as weight decay.
optimizer = AdamW(model.parameters(), lr=1e-5)

labels = [label2id[label] for label in train_labels]
labels = torch.tensor(labels).unsqueeze(0)
outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
loss = outputs.loss
loss.backward()
optimizer.step()

In [26]:
sentiment_analyzer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
sentiment_analyzer("地震怖い")

[{'label': 'POSITIVE', 'score': 0.5616199374198914}]

In [17]:
# 学習データに対する推論
_ = list(map(lambda x: print(f"{x}: {sentiment_analyzer(x)}"), train_docs))

サンプルコード見ながらバグってる: [{'label': 'POSITIVE', 'score': 0.5391868948936462}]
python多めの後にR触るとdata.trainとか書いてあるとき頭がバグる: [{'label': 'POSITIVE', 'score': 0.535291314125061}]
痩せる: [{'label': 'POSITIVE', 'score': 0.5796484351158142}]
ホームページの写真、新しくしてもらったんだけど顔丸すぎて萎えた: [{'label': 'POSITIVE', 'score': 0.5957570672035217}]
はい！再来週やります！
無料なのでお気軽にご参加ください😊 https://t.co/VfSozauXFf: [{'label': 'POSITIVE', 'score': 0.5378047227859497}]
あほすぎていろいろミスったごめんなさいいいいいいいい: [{'label': 'POSITIVE', 'score': 0.5752238035202026}]
こないだ楕円曲線やった流れでmodular formの勉強ちゃんとしてみたい欲がきてる
これとか読めたらいいなって思うけどなかなか時間がない🥺
 https://t.co/PEH0W6jMGm: [{'label': 'POSITIVE', 'score': 0.5618516206741333}]
数学会久しぶりに行きたいな
お祭りみあって楽しい🤤: [{'label': 'POSITIVE', 'score': 0.5314012765884399}]
現代数学レクチャーシリーズのお手伝いしてきました！
去年東工大のレクチャーシアターでしていただいたお話のパワーアップ版、楽しかったーー！
ご参加いただいた方、小野寺先生ありがとうございました🙏
#すうがくぶんか: [{'label': 'POSITIVE', 'score': 0.5564006567001343}]
励みになります🙏: [{'label': 'POSITIVE', 'score': 0.5632424354553223}]
確認がてら各ページ見てるんですけど、めっちゃいい感じになりますよ！ https://t.co/Wz